In [1]:
from modules.llm import *
from modules.utils import *

In [2]:
config = {
    "rqa_prompt_template" : "This database is a list of dataset metadata. Use the following pieces of context to find the relevant document. Answer only from the context given using the {question} given. If you do not know the answer, say you do not know. {context}",
    "num_return_documents" : 100,
    "embedding_model": "BAAI/bge-base-en-v1.5",
    "llm_model": "HuggingFaceH4/zephyr-7b-beta",
    "recreate_chroma": False,
}

In [3]:
metadata_df = create_metadata_dataframe(*get_all_dataset_metadata_from_openml())
metadata_df = clean_metadata_dataframe(metadata_df)

print(metadata_df.loc[20]['description'][:300])

**Author**: Mary McLeish & Matt Cecile, University of Guelph  
Donor: Will Taylor (taylor@pluto.arc.nasa.gov)   
**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Horse+Colic) - 8/6/89   

**Horse Colic database**  
Database of surgeries on horses. Possible class attributes: 24 (whether lesi


In [4]:
vectordb = load_document_and_create_vector_store(metadata_df, model_name=config['embedding_model'], recreate_chroma=config['recreate_chroma'])

/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
retriever, llm = create_retriever_and_llm(vectordb,num_return_documents=config["num_return_documents"], model_repo_id=config["llm_model"])
qa = create_llm_chain_and_query(vectordb=vectordb,retriever=retriever,llm=llm, prompt_template = config["rqa_prompt_template"])

query = "Which datasets would be useful for stock market information?"


/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [8]:
%time
results = create_result_dataframe(query, qa)
results

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


,did,description,short_description
0,43006,This datasets covers features from various cat...,This datasets covers features from various cat...
1,43004,This datasets covers features from various cat...,This datasets covers features from various cat...
2,43003,This datasets covers features from various cat...,This datasets covers features from various cat...
3,43005,This datasets covers features from various cat...,This datasets covers features from various cat...
4,43555,Check the movement of the financial market thr...,Check the movement of the financial market thr...
...,...,...,...
90,40981,**Author**: Confidential. Donated by Ross Quin...,**Author**: Confidential. Donated by Ross Quin...
91,664,More information about the data sets and the b...,More information about the data sets and the b...
92,1030,3. Past Usage:\n\n4. Relevant Information\nThe...,3. Past Usage:\n\n4. Relevant Information\nThe...
93,43545,Context\nThis dataset was created when I pract...,Context\nThis dataset was created when I pract...


In [7]:
results['short_description'].values

array(['This datasets covers features from various categories of technical indicators, futures contracts, price of commodities, important indices of markets around the world, price of major companies in the U.S. market, and treasury bill rates. Sources and thorough description of features have been mentione',
       'This datasets covers features from various categories of technical indicators, futures contracts, price of commodities, important indices of markets around the world, price of major companies in the U.S. market, and treasury bill rates. Sources and thorough description of features have been mentione',
       'This datasets covers features from various categories of technical indicators, futures contracts, price of commodities, important indices of markets around the world, price of major companies in the U.S. market, and treasury bill rates. Sources and thorough description of features have been mentione',
       'This datasets covers features from various categories of te